In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/casml-generative-ai-hackathon/Dataset_RAG (1)/queries.json
/kaggle/input/casml-generative-ai-hackathon/Dataset_RAG (1)/book.pdf
/kaggle/input/qwen2.5/transformers/3b-instruct/1/model.safetensors.index.json
/kaggle/input/qwen2.5/transformers/3b-instruct/1/config.json
/kaggle/input/qwen2.5/transformers/3b-instruct/1/merges.txt
/kaggle/input/qwen2.5/transformers/3b-instruct/1/model-00001-of-00002.safetensors
/kaggle/input/qwen2.5/transformers/3b-instruct/1/LICENSE
/kaggle/input/qwen2.5/transformers/3b-instruct/1/model-00002-of-00002.safetensors
/kaggle/input/qwen2.5/transformers/3b-instruct/1/README.md
/kaggle/input/qwen2.5/transformers/3b-instruct/1/tokenizer.json
/kaggle/input/qwen2.5/transformers/3b-instruct/1/vocab.json
/kaggle/input/qwen2.5/transformers/3b-instruct/1/tokenizer_config.json
/kaggle/input/qwen2.5/transformers/3b-instruct/1/.gitattributes
/kaggle/input/qwen2.5/transformers/3b-instruct/1/generation_config.json


In [2]:
# !pip install -q pdfplumber faiss-cpu sentence-transformers transformers accelerate
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 36.6 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 46.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 67.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# **建立一個帶有 metadata 的 Knowledge QA 系統，主要應用在 CASML Hackathon 的 PDF 文件知識問答。**

In [5]:
# -*- coding: utf-8 -*-
"""
Knowledge QA for CASML Hackathon with metadata, references, and improved prompts.
Includes log suppression and environment setup to avoid CUDA/XLA warnings.
"""

# ===== 環境變數設置：抑制常見警告與 CUDA/XLA 初始化錯誤 =====
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # 隱藏 TensorFlow/XLA 初始化日誌（cuDNN/cuBLAS 重複註冊警告）
os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir=/nonexistent"  # 阻止 XLA 掃描 CUDA plugins，避免註冊衝突
os.environ["QWEN_ENABLE_WINDOW_ATTENTION"] = "0"  # 關閉 Sliding Window Attention，避免 SDPA 未實作警告

# ===== 日誌層級控制：抑制 transformers 模型初始化時的冗長輸出 =====
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

import json
import csv
import faiss  # 高效向量資料庫搜尋工具
import numpy as np
import time
from tqdm import tqdm
from typing import List
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer # 做句子編碼（轉成向量）。
from transformers import AutoTokenizer, AutoModelForCausalLM

def extract_chunks_with_metadata(pdf_path: str, chunk_size=1500, stride=100):
    """
    從 PDF 擷取每一頁的純文字，並依 chunk + stride 切割，附上 page/section metadata。
    """
    reader = PdfReader(pdf_path)
    chunks = []
    for i, page in enumerate(reader.pages):
        page_num = str(i + 1)
        text = page.extract_text()
        if not text:
            continue
        section = f"page_{page_num}"
        start = 0
        while start < len(text):
            end = min(start + chunk_size, len(text))
            chunk_text = text[start:end]
            if len(chunk_text.strip()) > 50:
                chunks.append({
                    "text": chunk_text.strip(),
                    "page": page_num,
                    "sections": [section]
                })
            start += (chunk_size - stride)
    return chunks

def create_vector_db_with_metadata(chunks: List[dict], encoder_model: SentenceTransformer):
    """
    將 chunks 編碼為向量並建立 FAISS 資料庫，並建立 id -> text/meta 對應表。
    """
    print("Encoding texts for FAISS...")
    texts = [chunk["text"] for chunk in chunks]
    vectors = encoder_model.encode(texts, show_progress_bar=True)  # 將文字轉換為數值向量。
    dimension = vectors.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(vectors, dtype=np.float32))
    """
    建立兩個 mapping：

    id_to_text：index ID ➜ chunk 原文

    id_to_meta：index ID ➜ 該 chunk 的 page/section
    """
    
    id_to_text = {i: chunk["text"] for i, chunk in enumerate(chunks)}
    id_to_meta = {i: {"sections": chunk["sections"], "page": chunk["page"]} for i, chunk in enumerate(chunks)}
    return index, id_to_text, id_to_meta

def build_references(indices: List[int], id_to_meta: dict):
    """
    根據 FAISS 檢索結果，彙整引用來源為 JSON 格式的 sections 與 pages。
    """
    sections = set()
    pages = set()
    for idx in indices:
        meta = id_to_meta.get(idx, {})
        for sec in meta.get("sections", []):
            sections.add(sec)
        pages.add(meta.get("page", ""))
    return json.dumps({
        "sections": sorted(list(sections)),
        "pages": sorted(list(pages))
    }, ensure_ascii=False)

def get_answer(query: str, index: faiss.Index, id_to_text_map: dict,
               id_to_meta_map: dict, encoder_model: SentenceTransformer,
               llm_model: AutoModelForCausalLM, llm_tokenizer: AutoTokenizer):
    """
    對 query 進行向量檢索與回答生成，回傳答案與引用 context/references。
    """
    print(f"Vectorizing query: '{query}'")
    query_vec = encoder_model.encode([query], convert_to_tensor=False, show_progress_bar=False)
    D, I = index.search(np.array(query_vec, dtype=np.float32), k=5)
    retrieved_indices = [i for i in I[0] if i >= 0 and i < len(id_to_text_map)]
    if not retrieved_indices:
        return "The information needed to answer this question was not found in the document.", "", "[]"
    retrieved_context = "\n---\n".join([id_to_text_map[i] for i in retrieved_indices])
    references_str = build_references(retrieved_indices, id_to_meta_map)
    prompt = f"""You are a helpful teaching assistant. Answer the question using only the context below.
If the context is insufficient, reply: \"The information needed to answer this question was not found in the document.\"

[Context]
{retrieved_context}

[Question]
{query}

[Answer]
"""
    inputs = llm_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(llm_model.device)
    outputs = llm_model.generate(**inputs, max_new_tokens=512, pad_token_id=llm_tokenizer.eos_token_id)
    answer = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.strip(), retrieved_context, references_str

def load_local_qwen_model(model_path: str):
    """
    從本地載入 Qwen2.5 模型與 tokenizer。
    """
    print("Loading Qwen2.5 model from:", model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True, device_map="auto", trust_remote_code=True)
    print("Qwen2.5 model loaded successfully.")
    return tokenizer, model

In [4]:
def main():

    """
    主流程：從 PDF 建立語意索引，載入模型，對 query 批次產生回答並存成 CSV。
    """
    
    pdf_path = "/kaggle/input/casml-generative-ai-hackathon/Dataset_RAG (1)/book.pdf"
    queries_path = "/kaggle/input/casml-generative-ai-hackathon/Dataset_RAG (1)/queries.json"
    answers_csv_path = "/kaggle/working/answers.csv"
    encoder_model_name = "BAAI/bge-small-zh-v1.5"
    qwen_model_path = "/kaggle/input/qwen2.5/transformers/3b-instruct/1"

    print(f"Extracting chunks from {pdf_path}...")
    chunks = extract_chunks_with_metadata(pdf_path)
    print(f"Extracted {len(chunks)} chunks.")

    print(f"Loading encoder model: {encoder_model_name}...")
    encoder_model = SentenceTransformer(encoder_model_name)
    print("Encoder model loaded.")

    print("Creating FAISS vector database...")
    vector_index, id_to_text_mapping, id_to_meta_mapping = create_vector_db_with_metadata(chunks, encoder_model)
    print("FAISS index created.")

    llm_tokenizer, llm_model = load_local_qwen_model(qwen_model_path)

    print(f"Loading queries from {queries_path}...")
    with open(queries_path, "r", encoding="utf-8") as f:
        queries_data = json.load(f)
    print(f"Loaded {len(queries_data)} queries.")

    print("Processing queries and saving to CSV...")
    with open(answers_csv_path, "w", encoding="utf-8", newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["ID", "context", "answer", "references"])
        writer.writeheader()

        for q_data in tqdm(queries_data, desc="Answering queries"):
            query_id = q_data.get("id") or q_data.get("query_id")
            if query_id is None or "question" not in q_data:
                continue
            question_text = q_data["question"]

            print(f"\nQuery ID: {query_id}")
            print(f"Question: {question_text}")
            start_time = time.time()

            answer_text, retrieved_context, references_str = get_answer(
                question_text, vector_index, id_to_text_mapping, id_to_meta_mapping,
                encoder_model, llm_model, llm_tokenizer
            )

            duration = time.time() - start_time
            print(f"Finished in {duration:.2f} seconds")

            writer.writerow({
                "ID": int(query_id),
                "context": retrieved_context,
                "answer": answer_text,
                "references": references_str
            })
    print(f"All answers saved to {answers_csv_path}.")

if __name__ == "__main__":
    main()

Extracting chunks from /kaggle/input/casml-generative-ai-hackathon/Dataset_RAG (1)/book.pdf...
Extracted 2098 chunks.
Loading encoder model: BAAI/bge-small-zh-v1.5...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/27.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

Finished in 502.36 seconds

Query ID: 49
Question: What is the impact of cultural diversity on psychological research?
Vectorizing query: 'What is the impact of cultural diversity on psychological research?'


Answering queries:  98%|█████████▊| 49/50 [3:40:27<05:08, 308.09s/it]

Finished in 241.98 seconds

Query ID: 50
Question: What is the role of critical thinking in psychology?
Vectorizing query: 'What is the role of critical thinking in psychology?'


Answering queries: 100%|██████████| 50/50 [3:42:53<00:00, 267.48s/it]

Finished in 146.42 seconds
All answers saved to /kaggle/working/answers.csv.
